# AirBnb Listings - Price Prediction Analysis

## Code 02 - Exploratory modeling

### Step 1: Getting the data ready

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 1
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import pickle
import string as str
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.model_selection as cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn import metrics
from sklearn.metrics import mean_squared_error as MSE

# Importing Regressor Modules
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
ny = pd.read_csv('cleaned_listings.csv')
ny.head(5)

,id,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_verifications_count,desc_len,host_about_len,lreview_year,lreview_month,lreview_day,freview_year,freview_month,freview_day,ndays_between_f_l_reviews,ndays_host,num_amenities,host_response_time_na,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,property_type_Condominium,property_type_House,property_type_Loft,property_type_Other,property_type_Serviced apartment,property_type_Townhouse,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_moderate,cancellation_policy_strict
0,2454,60,0,1.0,1.0,1,0,1,2,1.0,0.0,2.0,137.0,150.0,50.0,1,0.0,7,730,7,7,730,730,7.0,730.0,16,36,65,65,1,1,96.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,1,1,0,0,1.00,2,412,0,2019.0,1.0,30.0,2019.0,1.0,30.0,0.0,3803,15,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,2539,100,0,8.0,8.0,1,1,0,4,1.0,1.0,1.0,149.0,100.0,25.0,1,25.0,1,730,1,1,730,730,1.0,730.0,30,60,90,365,9,2,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,8,0,5,3,0.23,4,392,431,2018.0,10.0,19.0,2015.0,12.0,4.0,15.0,3800,44,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
2,2595,60,0,5.0,5.0,1,1,0,2,1.0,0.0,1.0,225.0,350.0,100.0,2,0.0,1,1125,1,1,1125,1125,1.0,1125.0,30,60,90,365,43,14,95.0,9.0,9.0,10.0,10.0,10.0,9.0,0,1,1,2,1,0,1,0.38,5,1000,427,2019.0,1.0,2.0,2009.0,11.0,21.0,19.0,3798,36,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,3330,100,0,4.0,4.0,1,1,1,2,1.0,1.0,1.0,70.0,150.0,125.0,2,50.0,5,730,5,5,730,730,5.0,730.0,2,27,37,290,39,12,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,3,0,3,0,0.35,4,1000,25,2018.0,12.0,7.0,2010.0,1.0,4.0,3.0,3737,25,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1
4,3647,100,0,1.0,1.0,1,1,1,2,1.0,1.0,1.0,150.0,200.0,75.0,2,20.0,3,7,3,3,7,7,3.0,7.0,30,60,90,365,0,0,96.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,1,1,0,1,0,0.76,6,1000,354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3721,11,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1


In [7]:
# Checking if Nulls exist
ny.isnull().any()

id                                              False
host_response_rate                              False
host_is_superhost                               False
host_listings_count                             False
host_total_listings_count                       False
host_has_profile_pic                            False
host_identity_verified                          False
is_location_exact                               False
accommodates                                    False
bathrooms                                       False
bedrooms                                        False
beds                                            False
price                                           False
security_deposit                                False
cleaning_fee                                    False
guests_included                                 False
extra_people                                    False
minimum_nights                                  False
maximum_nights              

In [4]:
ny.shape

(50220, 80)

In [5]:
# Setting ID as the index
ny.set_index('id', inplace=True)
ny.head()

,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,host_verifications_count,desc_len,host_about_len,lreview_year,lreview_month,lreview_day,freview_year,freview_month,freview_day,ndays_between_f_l_reviews,ndays_host,num_amenities,host_response_time_na,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,property_type_Condominium,property_type_House,property_type_Loft,property_type_Other,property_type_Serviced apartment,property_type_Townhouse,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_moderate,cancellation_policy_strict
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2454,60,0,1.0,1.0,1,0,1,2,1.0,0.0,2.0,137.0,150.0,50.0,1,0.0,7,730,7,7,730,730,7.0,730.0,16,36,65,65,1,1,96.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,1,1,0,0,1.00,2,412,0,2019.0,1.0,30.0,2019.0,1.0,30.0,0.0,3803,15,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2539,100,0,8.0,8.0,1,1,0,4,1.0,1.0,1.0,149.0,100.0,25.0,1,25.0,1,730,1,1,730,730,1.0,730.0,30,60,90,365,9,2,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,8,0,5,3,0.23,4,392,431,2018.0,10.0,19.0,2015.0,12.0,4.0,15.0,3800,44,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
2595,60,0,5.0,5.0,1,1,0,2,1.0,0.0,1.0,225.0,350.0,100.0,2,0.0,1,1125,1,1,1125,1125,1.0,1125.0,30,60,90,365,43,14,95.0,9.0,9.0,10.0,10.0,10.0,9.0,0,1,1,2,1,0,1,0.38,5,1000,427,2019.0,1.0,2.0,2009.0,11.0,21.0,19.0,3798,36,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3330,100,0,4.0,4.0,1,1,1,2,1.0,1.0,1.0,70.0,150.0,125.0,2,50.0,5,730,5,5,730,730,5.0,730.0,2,27,37,290,39,12,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,3,0,3,0,0.35,4,1000,25,2018.0,12.0,7.0,2010.0,1.0,4.0,3.0,3737,25,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1
3647,100,0,1.0,1.0,1,1,1,2,1.0,1.0,1.0,150.0,200.0,75.0,2,20.0,3,7,3,3,7,7,3.0,7.0,30,60,90,365,0,0,96.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,1,1,0,1,0,0.76,6,1000,354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3721,11,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1


In [9]:
# Outlier removal
ny_new = ny[ny['price'] <= 500.0]
ny_new.shape

(49174, 79)

In [11]:
# Setting X and Y variables 
X = ny_new.drop(ny_new.columns[[11]],axis=1)
Y = ny_new.loc[:,'price']

In [12]:
Y.head()

id
2454    137.0
2539    149.0
2595    225.0
3330     70.0
3647    150.0
Name: price, dtype: float64

In [13]:
X.shape

(49174, 78)

### Step 2: Standardizing the variables

In [14]:
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)
X_std = pd.DataFrame(X_sc)
X_std.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
0,-2.745641,-0.456979,-0.124279,-0.124279,0.042581,-1.003749,0.464989,-0.465314,-0.325703,-1.638064,0.479360,-0.198032,-0.236363,-0.460228,-0.603527,0.008798,-0.004584,0.016981,-0.046925,-0.004584,-0.004585,-0.041685,-0.004584,0.426902,0.530235,0.822304,-0.414175,-0.512008,-0.525973,0.230844,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,-0.154608,-0.161370,-0.226565,-0.148009,-0.296182,-0.105451,-0.168218,-1.676303,-1.117645,-0.533582,0.502674,-0.756091,1.670895,0.504576,-1.116921,1.734453,-0.921514,2.940892,-0.544759,-0.919072,3.316993,-0.369172,-0.688153,-0.838554,1.110428,-0.356213,-0.083688,5.951906,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,-0.915614,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,-0.552275,-0.913964
1,0.287659,-0.456979,-0.039020,-0.039020,0.042581,0.996265,-2.150588,0.681074,-0.325703,-0.221399,-0.505682,-0.326899,-0.788652,-0.460228,0.476267,-0.299806,-0.004584,-0.291038,-0.091493,-0.004584,-0.004585,-0.135773,-0.004584,1.565478,1.507142,1.506688,1.779090,-0.320404,-0.463309,0.496656,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,-0.154608,-0.161370,0.174273,-0.206749,0.828416,2.529886,-0.696562,-0.657661,-1.180955,0.792456,0.501438,1.183399,0.654114,0.499625,1.532268,-0.792860,0.884153,2.937121,2.439606,-0.919072,-0.301478,-0.369172,1.453165,1.192529,-0.900554,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,1.092163,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,1.810692,-0.913964
2,-2.745641,-0.456979,-0.075559,-0.075559,0.042581,0.996265,-2.150588,-0.465314,-0.325703,-1.638064,-0.505682,0.317436,0.868213,0.484749,-0.603527,-0.299806,-0.004544,-0.291038,-0.091493,-0.004543,-0.004544,-0.135773,-0.004544,1.565478,1.507142,1.506688,1.779090,0.493913,0.288653,0.097938,-0.925535,-0.425149,0.31912,0.305719,0.519257,-0.615978,-0.762637,6.467959,6.196954,-0.169303,-0.148009,-0.296182,0.772994,-0.593638,-0.148340,0.743684,0.780150,0.502674,-0.756091,-0.917275,0.492199,1.291433,0.859614,1.365664,2.934607,1.616333,-0.919072,3.316993,-0.369172,-0.688153,-0.838554,1.110428,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,-0.915614,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,-0.552275,1.094135
3,0.287659,-0.456979,-0.087739,-0.087739,0.042581,0.996265,0.464989,-0.465314,-0.325703,-0.221399,-0.505682,-0.198032,1.420502,0.484749,1.556060,-0.094070,-0.004584,-0.085692,-0.061781,-0.004584,-0.004585,-0.073048,-0.004584,-0.711674,0.163895,0.055794,1.230774,0.398111,0.163326,0.363750,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,-0.154608,-0.161370,-0.112040,-0.206749,0.378577,-0.105451,-0.614223,-0.657661,0.743684,-0.456665,0.501438,1.614397,-0.455102,0.493437,-1.116921,-0.792860,-0.560381,2.857936,0.484332,1.088054,-0.301478,-0.369172,-0.688153,1.192529,-0.900554,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,1.092163,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,-0.552275,1.094135
4,0.287659,-0.456979,-0.124279,-0.124279,0.042581,0.996265,0.464989,-0.465314,-0.325703,-0.221399,-0.505682,-0.069165,0.315925,0.484749,0.260308,-0.196938,-0.004659,-0.188365,-0.076637,-0.004659,-0.004659,-0.104410,-0.004659,1.565478,1.507142,1.506688,1.779090,-0.535959,-0.588637,0.230844,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,6.467959,6.196954,-0.226565,-0.206749,-0.071262,-0.105451,-0.332897,0.360980,0.743684,0.555554,-1.994272,-0.971590,-1.102144,-1.994270,-1.357756,-1.181677,-0.921514,2.837826,-0.956395,1.088054,-0.301478,-0.369172,-0.688153,-0.838554,1.110428,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,1.092163,-0.153848,-0.039859,-0.082571,13.190771,-7.416659,-0.552275,1.094135


In [15]:
rowNamesArr = ny.index
listOfrowNames = list(rowNamesArr)

X_std.rename(index = dict(zip(X_std.index[0:], listOfrowNames)),inplace=True)

In [16]:
X_std.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
2454,-2.745641,-0.456979,-0.124279,-0.124279,0.042581,-1.003749,0.464989,-0.465314,-0.325703,-1.638064,0.479360,-0.198032,-0.236363,-0.460228,-0.603527,0.008798,-0.004584,0.016981,-0.046925,-0.004584,-0.004585,-0.041685,-0.004584,0.426902,0.530235,0.822304,-0.414175,-0.512008,-0.525973,0.230844,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,-0.154608,-0.161370,-0.226565,-0.148009,-0.296182,-0.105451,-0.168218,-1.676303,-1.117645,-0.533582,0.502674,-0.756091,1.670895,0.504576,-1.116921,1.734453,-0.921514,2.940892,-0.544759,-0.919072,3.316993,-0.369172,-0.688153,-0.838554,1.110428,-0.356213,-0.083688,5.951906,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,-0.915614,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,-0.552275,-0.913964
2539,0.287659,-0.456979,-0.039020,-0.039020,0.042581,0.996265,-2.150588,0.681074,-0.325703,-0.221399,-0.505682,-0.326899,-0.788652,-0.460228,0.476267,-0.299806,-0.004584,-0.291038,-0.091493,-0.004584,-0.004585,-0.135773,-0.004584,1.565478,1.507142,1.506688,1.779090,-0.320404,-0.463309,0.496656,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,-0.154608,-0.161370,0.174273,-0.206749,0.828416,2.529886,-0.696562,-0.657661,-1.180955,0.792456,0.501438,1.183399,0.654114,0.499625,1.532268,-0.792860,0.884153,2.937121,2.439606,-0.919072,-0.301478,-0.369172,1.453165,1.192529,-0.900554,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,1.092163,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,1.810692,-0.913964
2595,-2.745641,-0.456979,-0.075559,-0.075559,0.042581,0.996265,-2.150588,-0.465314,-0.325703,-1.638064,-0.505682,0.317436,0.868213,0.484749,-0.603527,-0.299806,-0.004544,-0.291038,-0.091493,-0.004543,-0.004544,-0.135773,-0.004544,1.565478,1.507142,1.506688,1.779090,0.493913,0.288653,0.097938,-0.925535,-0.425149,0.31912,0.305719,0.519257,-0.615978,-0.762637,6.467959,6.196954,-0.169303,-0.148009,-0.296182,0.772994,-0.593638,-0.148340,0.743684,0.780150,0.502674,-0.756091,-0.917275,0.492199,1.291433,0.859614,1.365664,2.934607,1.616333,-0.919072,3.316993,-0.369172,-0.688153,-0.838554,1.110428,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,-0.915614,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,-0.552275,1.094135
3330,0.287659,-0.456979,-0.087739,-0.087739,0.042581,0.996265,0.464989,-0.465314,-0.325703,-0.221399,-0.505682,-0.198032,1.420502,0.484749,1.556060,-0.094070,-0.004584,-0.085692,-0.061781,-0.004584,-0.004585,-0.073048,-0.004584,-0.711674,0.163895,0.055794,1.230774,0.398111,0.163326,0.363750,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,-0.154608,-0.161370,-0.112040,-0.206749,0.378577,-0.105451,-0.614223,-0.657661,0.743684,-0.456665,0.501438,1.614397,-0.455102,0.493437,-1.116921,-0.792860,-0.560381,2.857936,0.484332,1.088054,-0.301478,-0.369172,-0.688153,1.192529,-0.900554,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,1.092163,-0.153848,-0.039859,-0.082571,-0.075811,0.134832,-0.552275,1.094135
3647,0.287659,-0.456979,-0.124279,-0.124279,0.042581,0.996265,0.464989,-0.465314,-0.325703,-0.221399,-0.505682,-0.069165,0.315925,0.484749,0.260308,-0.196938,-0.004659,-0.188365,-0.076637,-0.004659,-0.004659,-0.104410,-0.004659,1.565478,1.507142,1.506688,1.779090,-0.535959,-0.588637,0.230844,0.408236,0.573908,0.31912,0.305719,0.519257,0.579558,-0.762637,6.467959,6.196954,-0.226565,-0.206749,-0.071262,-0.105451,-0.332897,0.360980,0.743684,0.555554,-1.994272,-0.971590,-1.102144,-1.994270,-1.357756,-1.181677,-0.921514,2.837826,-0.956395,1.088054,-0.301478,-0.369172,-0.688153,-0.838554,1.110428,-0.356213,-0.083688,-0.168013,-0.282947,-0.170493,-0.147207,-0.118062,-0.180587,1.092163,-0.153848,-0.039859,-0.082571,13.190771,-7.416659,-0.552275,1.094135


### Step 3: Splitting the dataset into Train and Test

In [17]:
X_train, X_test, Y_train, Y_test = cv.train_test_split(X_std, Y, test_size = 0.3, random_state = 0)

### Step 4: Running Decision Trees

#### a. Fit the model

In [49]:
dtree = DecisionTreeRegressor(random_state=0)

# Fit the model with train data
dtree.fit(X_train,Y_train)

# Predict on train data
Y_pred2 = dtree.predict(X_train)

# Predict on test data
Y_pred = dtree.predict(X_test)

#### b. Look at model evaluation parameters

In [50]:
# Train Data

# Compute mse_dt
mse_dt = MSE(Y_train, Y_pred2)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Train set RMSE: {:.2f}".format(rmse_dt))

Train set RMSE: 1.82


In [45]:
# Use cross-validation on Train data

CV_scores = - cv.cross_val_score(dtree, X_train, Y_train, scoring='neg_mean_squared_error', cv=10)

# Compute the 10-folds CV
CV = CV_scores.mean()**(1/2)

# Print Train CV accuracy
print('Cross Validation RMSE: {:.2f}'.format(CV))

Cross Validation RMSE: 311.57


In [46]:
# Test Data

# Compute mse_dt
mse_dt = MSE(Y_test, Y_pred)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Test set RMSE: {:.2f}".format(rmse_dt))

Test set RMSE: 274.70


### Step 5: Running Random Forests

#### a. Fit the model

In [51]:
rf = RandomForestRegressor(random_state=0)

# Fit the model with train data
rf.fit(X_train,Y_train)

# Predict on train data
Y_pred2 = rf.predict(X_train)

# Predict on test data
Y_pred = rf.predict(X_test)

#### b. Look at model evaluation parameters

In [52]:
# Train Data

# Compute mse_dt
mse_dt = MSE(Y_train, Y_pred2)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Train set RMSE: {:.2f}".format(rmse_dt))

Train set RMSE: 90.72


In [54]:
# Use cross-validation on Train data

CV_scores = - cv.cross_val_score(rf, X_train, Y_train, scoring='neg_mean_squared_error', cv=10)

# Compute the 10-folds CV
CV = CV_scores.mean()**(1/2)

# Print Train CV accuracy
print('Cross Validation RMSE: {:.2f}'.format(CV))

Cross Validation RMSE: 224.89


In [53]:
# Test Data

# Compute mse_dt
mse_dt = MSE(Y_test, Y_pred)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Test set RMSE: {:.2f}".format(rmse_dt))

Test set RMSE: 190.01


### Step 6: Running Gradient Boosting

#### a. Fit the model

In [18]:
gb = GradientBoostingRegressor(n_estimators=400, random_state=0)

# Fit the model with train data
gb.fit(X_train,Y_train)

# Predict on train data
Y_pred2 = gb.predict(X_train)

# Predict on test data
Y_pred = gb.predict(X_test)

#### b. Look at model evaluation parameters

In [19]:
# Train Data

# Compute mse_dt
mse_dt = MSE(Y_train, Y_pred2)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Train set RMSE: {:.2f}".format(rmse_dt))

Train set RMSE: 49.23


In [ ]:
# Use cross-validation on Train data

CV_scores = - cv.cross_val_score(gb, X_train, Y_train, scoring='neg_mean_squared_error', cv=10)

# Compute the 10-folds CV
CV = CV_scores.mean()**(1/2)

# Print Train CV accuracy
print('Cross Validation RMSE: {:.2f}'.format(CV))

In [20]:
# Test Data

# Compute mse_dt
mse_dt = MSE(Y_test, Y_pred)

# Compute rmse_dt
rmse_dt = mse_dt**(1/2)

# Print rmse_dt
print("Test set RMSE: {:.2f}".format(rmse_dt))

Test set RMSE: 52.11
